In [1]:
import numpy as np                        # ndarrys for gridded data
import pandas as pd                       # DataFrames for tabular data
import matplotlib.pyplot as plt           # plotting
from scipy import stats                   # summary statistics
import math                               # trigonometry etc.
import random                             # randon numbers
from scipy.stats import norm              # Gaussian parametric distribution
import geostatspy.GSLIB as GSLIB
from ipywidgets import interactive                      # widgets and interactivity
from ipywidgets import widgets                            
from ipywidgets import Layout
from ipywidgets import Label
from ipywidgets import VBox, HBox
import os                                                 # to set current working directory 
import math                                               # basic calculations like square root
from sklearn.model_selection import train_test_split      # train and test split
from sklearn import svm                                   # support vector machine methods
from sklearn import tree                    

# tree program from scikit learn (package for machine learning)
from sklearn.metrics import mean_squared_error, r2_score  # specific measures to check our models
import pandas.plotting as pd_plot
from intake import cat                                    # data catalogue
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from tqdm.notebook import trange, tqdm



well_df = pd.read_csv('wellData.csv')
day_df = pd.read_csv('dailyData.csv')

well_df = well_df.drop(['ESP_Motor_Frequency_Rating', 'Unnamed: 0', 'Artificial_Lift_Type'], axis =1)
well_df

,Well_ID,AL_Key,AL_Bottom_Depth,ESP_Pump_Stages,DLS_Critical,ESP_Motor_Current_Rating,ESP_Motor_Voltage_Rating,ESP_Motor_Power_Rating,DLS_at_Set_Depth
0,345,ESP_2,9056.67,0,5.323174,NaN,NaN,NaN,1.713554
1,29,ESP_3,8982.09,0,6.440650,NaN,NaN,NaN,2.789395
2,327,ESP_1,8876.37,0,5.669751,NaN,NaN,NaN,0.481680
3,71,ESP_1,8930.11,0,5.614798,NaN,NaN,NaN,0.842038
4,125,ESP_2,8616.41,348,26.061750,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
161,341,ESP_1,7927.90,504,17.561921,NaN,NaN,NaN,NaN
162,143,ESP_1,8041.24,0,12.231519,NaN,NaN,NaN,NaN
163,258,ESP_1,7908.00,445,12.905506,NaN,NaN,NaN,9.620484
164,285,ESP_1,7958.81,415,12.924289,NaN,NaN,NaN,10.935088


In [2]:
day_df = day_df.drop('ARTIFICIAL_LIFT', axis =1)
"""
day_df = day_df.drop(['Lower_Limit', 'Power_Difference', 'ESP Data - Motor Winding Temperature', 'Drive_Power','ESP Data - Output Frequency',
'Gas_through_Annulus_Intake','ESP Data - Drive Voltage','Discharge_Pressure','ESP Data - Motor Temperature Shutdown Setpoint',
'Gas_Saturation_at_Discharge','Pump_Average_Pressure','Pump_Power','ARTIFICIAL_LIFT',
'Power_Ratio','ESP Data - Drive Current','ESP Data - Vibration X','Startup_Count','Gas_through_Annulus','ESP_Fluid','Pb_ESP'], axis = 1)
day_df
"""

"\nday_df = day_df.drop(['Lower_Limit', 'Power_Difference', 'ESP Data - Motor Winding Temperature', 'Drive_Power','ESP Data - Output Frequency',\n'Gas_through_Annulus_Intake','ESP Data - Drive Voltage','Discharge_Pressure','ESP Data - Motor Temperature Shutdown Setpoint',\n'Gas_Saturation_at_Discharge','Pump_Average_Pressure','Pump_Power','ARTIFICIAL_LIFT',\n'Power_Ratio','ESP Data - Drive Current','ESP Data - Vibration X','Startup_Count','Gas_through_Annulus','ESP_Fluid','Pb_ESP'], axis = 1)\nday_df\n"

In [3]:
new_df_2 = well_df.drop(['AL_Key'], axis = 1).copy()

imputer_2 = IterativeImputer(tol = 1e-2)
imputer_2.fit(new_df_2)
imputed_array_2 = imputer_2.transform(new_df_2)

imputed2_df = pd.DataFrame(imputed_array_2)
imputed2_df

,0,1,2,3,4,5,6,7
0,345.0,9056.67,0.0,5.323174,56.046929,3409.779481,309.252209,1.713554
1,29.0,8982.09,0.0,6.440650,54.279085,3739.342582,335.712037,2.789395
2,327.0,8876.37,0.0,5.669751,58.025138,3385.829184,307.680447,0.481680
3,71.0,8930.11,0.0,5.614798,55.299631,3679.213287,330.892139,0.842038
4,125.0,8616.41,348.0,26.061750,64.887168,2297.913622,233.736284,4.789396
...,...,...,...,...,...,...,...,...
161,341.0,7927.90,504.0,17.561921,78.077368,1324.215271,160.627653,5.899696
162,143.0,8041.24,0.0,12.231519,66.005019,3394.013203,310.617004,4.213202
163,258.0,7908.00,445.0,12.905506,76.710015,1623.538707,182.290634,9.620484
164,285.0,7958.81,415.0,12.924289,75.754476,1715.839933,188.750968,10.935088


In [4]:
names_well = new_df_2.columns

i = 0
static_df = imputed2_df.rename({i : names_well[i] for i in range(len(names_well))}, axis = 'columns')
    
static_df["AL_Key"] = well_df.loc[:,"AL_Key"]
static_df

,Well_ID,AL_Bottom_Depth,ESP_Pump_Stages,DLS_Critical,ESP_Motor_Current_Rating,ESP_Motor_Voltage_Rating,ESP_Motor_Power_Rating,DLS_at_Set_Depth,AL_Key
0,345.0,9056.67,0.0,5.323174,56.046929,3409.779481,309.252209,1.713554,ESP_2
1,29.0,8982.09,0.0,6.440650,54.279085,3739.342582,335.712037,2.789395,ESP_3
2,327.0,8876.37,0.0,5.669751,58.025138,3385.829184,307.680447,0.481680,ESP_1
3,71.0,8930.11,0.0,5.614798,55.299631,3679.213287,330.892139,0.842038,ESP_1
4,125.0,8616.41,348.0,26.061750,64.887168,2297.913622,233.736284,4.789396,ESP_2
...,...,...,...,...,...,...,...,...,...
161,341.0,7927.90,504.0,17.561921,78.077368,1324.215271,160.627653,5.899696,ESP_1
162,143.0,8041.24,0.0,12.231519,66.005019,3394.013203,310.617004,4.213202,ESP_1
163,258.0,7908.00,445.0,12.905506,76.710015,1623.538707,182.290634,9.620484,ESP_1
164,285.0,7958.81,415.0,12.924289,75.754476,1715.839933,188.750968,10.935088,ESP_1


In [5]:
new_df = day_df.drop(['AL_Key'], axis = 1).copy()

imputer = IterativeImputer(tol = 5e-2)
imputer.fit(new_df)
imputed_array = imputer.transform(new_df)

imputed_df = pd.DataFrame(imputed_array)
imputed_df

/opt/conda/lib/python3.10/site-packages/sklearn/impute/_iterative.py:785: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,615.154601,650.447788,526.488655,11.0,249.860279,370.149250,368.715449,-509.534789,187.439259,57.443323,...,3.314110e+14,1154.825236,-1.623917e+14,-1.461464e+14,96.303097,-3.842932e+13,1.541419e+14,5.379407e+15,1148.388862,345.0
1,1224.193640,1232.455787,809.414685,0.0,135.239185,464.211750,-261.284551,-2445.534789,188.548634,60.605921,...,3.204875e+14,1406.825236,-1.570392e+14,-1.413294e+14,92.427609,-3.716267e+13,1.490613e+14,5.202099e+15,1208.795112,345.0
2,1007.769762,1047.068544,667.933899,0.0,146.440357,424.086750,-200.909551,-1629.534789,190.544728,60.009729,...,4.747602e+14,1200.106486,-2.326330e+14,-2.093609e+14,88.067746,-5.505163e+13,2.208148e+14,7.706228e+15,1284.802925,345.0
3,1514.641876,1370.143160,1122.137652,0.0,207.901295,505.961750,222.215449,-2717.534789,179.779103,59.466028,...,-2.539874e+14,2033.418986,1.244541e+14,1.120040e+14,116.018918,2.945154e+13,-1.181316e+14,-4.122681e+15,1255.006050,345.0
4,1089.741764,1387.138130,1156.169599,0.0,153.452076,487.446125,303.371699,-2293.534789,186.111134,58.695276,...,9.932682e+13,1976.973674,-4.867024e+13,-4.380139e+13,98.926816,-1.151761e+13,4.619770e+13,1.612256e+15,1128.841987,345.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50051,130.568357,335.639084,433.161488,0.0,232.000000,433.000000,894.599976,240.000000,182.800003,55.000000,...,1.703463e+03,1746.331510,1.681018e-01,1.872407e+01,100.456000,2.071009e-01,8.173193e+01,1.686796e+02,600.000000,142.0
50052,135.061535,338.161005,421.979434,0.0,232.000000,433.000000,894.599976,240.000000,182.800003,55.000000,...,1.668437e+03,1728.818696,1.719572e-01,1.826407e+01,100.456000,2.020129e-01,8.219193e+01,1.687669e+02,600.000000,142.0
50053,135.757477,341.732930,428.363835,0.0,230.433113,431.364779,893.432975,240.000000,182.836947,55.000000,...,1.663311e+03,1725.088591,1.732141e-01,1.845284e+01,99.400729,2.062677e-01,8.094789e+01,1.686728e+02,600.000000,142.0
50054,64.788755,170.379740,221.002588,12.0,243.458937,423.202102,934.540501,240.000000,186.748578,54.836954,...,1.604082e+03,1736.581602,1.627178e-01,8.862240e+00,103.032334,9.557130e-02,9.417009e+01,1.727154e+02,600.000000,142.0


In [6]:
names = new_df.columns

i = 0
dynamic_df = imputed_df.rename({i : names[i] for i in range(len(names))}, axis = 'columns')
    
dynamic_df["AL_Key"] = day_df.loc[:,"AL_Key"]
dynamic_df

,OIL,GAS,WATER,DOWN_TIME_HOURS,ESP Data - Drive Current,ESP Data - Drive Voltage,ESP Data - Intake Pressure,ESP Data - Motor Temperature Shutdown Setpoint,ESP Data - Motor Winding Temperature,ESP Data - Output Frequency,...,Pump_Average_Pressure,Gas_Saturation_in_Pump,Pump_Power,Drive_Power,Power_Ratio,Power_Difference,ESP_Temperature,Lower_Limit,Well_ID,AL_Key
0,615.154601,650.447788,526.488655,11.0,249.860279,370.149250,368.715449,-509.534789,187.439259,57.443323,...,1154.825236,-1.623917e+14,-1.461464e+14,96.303097,-3.842932e+13,1.541419e+14,5.379407e+15,1148.388862,345.0,ESP_2
1,1224.193640,1232.455787,809.414685,0.0,135.239185,464.211750,-261.284551,-2445.534789,188.548634,60.605921,...,1406.825236,-1.570392e+14,-1.413294e+14,92.427609,-3.716267e+13,1.490613e+14,5.202099e+15,1208.795112,345.0,ESP_2
2,1007.769762,1047.068544,667.933899,0.0,146.440357,424.086750,-200.909551,-1629.534789,190.544728,60.009729,...,1200.106486,-2.326330e+14,-2.093609e+14,88.067746,-5.505163e+13,2.208148e+14,7.706228e+15,1284.802925,345.0,ESP_2
3,1514.641876,1370.143160,1122.137652,0.0,207.901295,505.961750,222.215449,-2717.534789,179.779103,59.466028,...,2033.418986,1.244541e+14,1.120040e+14,116.018918,2.945154e+13,-1.181316e+14,-4.122681e+15,1255.006050,345.0,ESP_2
4,1089.741764,1387.138130,1156.169599,0.0,153.452076,487.446125,303.371699,-2293.534789,186.111134,58.695276,...,1976.973674,-4.867024e+13,-4.380139e+13,98.926816,-1.151761e+13,4.619770e+13,1.612256e+15,1128.841987,345.0,ESP_2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50051,130.568357,335.639084,433.161488,0.0,232.000000,433.000000,894.599976,240.000000,182.800003,55.000000,...,1746.331510,1.681018e-01,1.872407e+01,100.456000,2.071009e-01,8.173193e+01,1.686796e+02,600.000000,142.0,ESP_1
50052,135.061535,338.161005,421.979434,0.0,232.000000,433.000000,894.599976,240.000000,182.800003,55.000000,...,1728.818696,1.719572e-01,1.826407e+01,100.456000,2.020129e-01,8.219193e+01,1.687669e+02,600.000000,142.0,ESP_1
50053,135.757477,341.732930,428.363835,0.0,230.433113,431.364779,893.432975,240.000000,182.836947,55.000000,...,1725.088591,1.732141e-01,1.845284e+01,99.400729,2.062677e-01,8.094789e+01,1.686728e+02,600.000000,142.0,ESP_1
50054,64.788755,170.379740,221.002588,12.0,243.458937,423.202102,934.540501,240.000000,186.748578,54.836954,...,1736.581602,1.627178e-01,8.862240e+00,103.032334,9.557130e-02,9.417009e+01,1.727154e+02,600.000000,142.0,ESP_1


In [7]:
index_day_df = dynamic_df.set_index(["Well_ID", "AL_Key"])


In [8]:
total_days = dynamic_df.groupby(["Well_ID", "AL_Key"], sort = False, as_index = False)["OIL"].count()

static_df["Days_ToDeath"] = total_days.loc[:,"OIL"]
static_df

,Well_ID,AL_Bottom_Depth,ESP_Pump_Stages,DLS_Critical,ESP_Motor_Current_Rating,ESP_Motor_Voltage_Rating,ESP_Motor_Power_Rating,DLS_at_Set_Depth,AL_Key,Days_ToDeath
0,345.0,9056.67,0.0,5.323174,56.046929,3409.779481,309.252209,1.713554,ESP_2,431.0
1,29.0,8982.09,0.0,6.440650,54.279085,3739.342582,335.712037,2.789395,ESP_3,407.0
2,327.0,8876.37,0.0,5.669751,58.025138,3385.829184,307.680447,0.481680,ESP_1,356.0
3,71.0,8930.11,0.0,5.614798,55.299631,3679.213287,330.892139,0.842038,ESP_1,367.0
4,125.0,8616.41,348.0,26.061750,64.887168,2297.913622,233.736284,4.789396,ESP_2,128.0
...,...,...,...,...,...,...,...,...,...,...
161,341.0,7927.90,504.0,17.561921,78.077368,1324.215271,160.627653,5.899696,ESP_1,37.0
162,143.0,8041.24,0.0,12.231519,66.005019,3394.013203,310.617004,4.213202,ESP_1,262.0
163,258.0,7908.00,445.0,12.905506,76.710015,1623.538707,182.290634,9.620484,ESP_1,398.0
164,285.0,7958.81,415.0,12.924289,75.754476,1715.839933,188.750968,10.935088,ESP_1,428.0


In [9]:
L60_daily_df = []
for _iter in tqdm(index_day_df.index.unique()):
    sub_df = index_day_df.loc[_iter]
    L60_daily_df.append(sub_df.tail(60))
    
L60_daily_df = pd.concat(L60_daily_df)
L60_daily_df

  0%|          | 0/165 [00:00<?, ?it/s]

/tmp/ipykernel_539/2758770964.py:3: PerformanceWarning: indexing past lexsort depth may impact performance.
  sub_df = index_day_df.loc[_iter]


OIL         GAS       WATER  DOWN_TIME_HOURS  \
Well_ID AL_Key                                                        
345.0   ESP_2   288.280109  577.874234  285.779169              6.0   
        ESP_2   348.151863  702.095092  352.320096              3.0   
        ESP_2   205.246123  474.816577  229.794533             10.0   
        ESP_2   288.351195  572.334901  300.289356              6.0   
        ESP_2   340.058815  659.323860  318.221371              4.0   
...                    ...         ...         ...              ...   
142.0   ESP_1   130.568357  335.639084  433.161488              0.0   
        ESP_1   135.061535  338.161005  421.979434              0.0   
        ESP_1   135.757477  341.732930  428.363835              0.0   
        ESP_1    64.788755  170.379740  221.002588             12.0   
        ESP_1     0.000000    0.000000    0.000000             24.0   

                ESP Data - Drive Current  ESP Data - Drive Voltage  \
Well_ID AL_Key                                                       
345.0   ESP_2                 215.109996                397.465961   
        ESP_2                 198.041231                398.821667   
        ESP_2                 194.706247                400.003670   
        ESP_2                 190.186667                391.473530   
        ESP_2                 202.353641                381.488755   
...                                  ...                       ...   
142.0   ESP_1                 232.000000                433.000000   
        ESP_1                 232.000000                433.000000   
        ESP_1                 230.433113                431.364779   
        ESP_1                 243.458937                423.202102   
        ESP_1                 369.099649                402.275272   

                ESP Data - Intake Pressure  \
Well_ID AL_Key                               
345.0   ESP_2                   822.468938   
        ESP_2                   840.591594   
        ESP_2                   831.423514   
        ESP_2                   832.989565   
        ESP_2                   845.083539   
...                                    ...   
142.0   ESP_1                   894.599976   
        ESP_1                   894.599976   
        ESP_1                   893.432975   
        ESP_1                   934.540501   
        ESP_1                  1527.854901   

                ESP Data - Motor Temperature Shutdown Setpoint  \
Well_ID AL_Key                                                   
345.0   ESP_2                                      -661.948796   
        ESP_2                                      -624.821807   
        ESP_2                                      -691.838919   
        ESP_2                                      -468.889981   
        ESP_2                                      -564.632459   
...                                                        ...   
142.0   ESP_1                                       240.000000   
        ESP_1                                       240.000000   
        ESP_1                                       240.000000   
        ESP_1                                       240.000000   
        ESP_1                                       240.000000   

                ESP Data - Motor Winding Temperature  \
Well_ID AL_Key                                         
345.0   ESP_2                             188.956811   
        ESP_2                             187.808472   
        ESP_2                             187.275934   
        ESP_2                             186.506637   
        ESP_2                             187.187105   
...                                              ...   
142.0   ESP_1                             182.800003   
        ESP_1                             182.800003   
        ESP_1                             182.836947   
        ESP_1                             186.748578   
        ESP_1                             224.843614   

 

In [10]:
B60_daily_df = []
for _iter in tqdm(index_day_df.index.unique()):
    sub_df = index_day_df.loc[_iter]
    B60_daily_df.append(sub_df.head(60))
    
B60_daily_df = pd.concat(B60_daily_df)

B60_daily_df.insert(35, "Will_Fail", 0)
B60_daily_df

  0%|          | 0/165 [00:00<?, ?it/s]

/tmp/ipykernel_539/3144267950.py:3: PerformanceWarning: indexing past lexsort depth may impact performance.
  sub_df = index_day_df.loc[_iter]


OIL          GAS        WATER  DOWN_TIME_HOURS  \
Well_ID AL_Key                                                           
345.0   ESP_2    615.154601   650.447788   526.488655             11.0   
        ESP_2   1224.193640  1232.455787   809.414685              0.0   
        ESP_2   1007.769762  1047.068544   667.933899              0.0   
        ESP_2   1514.641876  1370.143160  1122.137652              0.0   
        ESP_2   1089.741764  1387.138130  1156.169599              0.0   
...                     ...          ...          ...              ...   
142.0   ESP_1    440.702023   448.032036  1241.324560              0.0   
        ESP_1    315.274071   345.029087  1007.802839              4.5   
        ESP_1    469.995107   491.710249  1265.696462              0.0   
        ESP_1    463.619406   467.508853  1197.749869              0.0   
        ESP_1    443.361393   464.585895  1200.952073              0.0   

                ESP Data - Drive Current  ESP Data - Drive Voltage  \
Well_ID AL_Key                                                       
345.0   ESP_2                 249.860279                370.149250   
        ESP_2                 135.239185                464.211750   
        ESP_2                 146.440357                424.086750   
        ESP_2                 207.901295                505.961750   
        ESP_2                 153.452076                487.446125   
...                                  ...                       ...   
142.0   ESP_1                 272.302616                397.000000   
        ESP_1                 273.492194                396.704341   
        ESP_1                 270.548958                397.000000   
        ESP_1                 269.545613                397.000000   
        ESP_1                 270.045324                397.000000   

                ESP Data - Intake Pressure  \
Well_ID AL_Key                               
345.0   ESP_2                   368.715449   
        ESP_2                  -261.284551   
        ESP_2                  -200.909551   
        ESP_2                   222.215449   
        ESP_2                   303.371699   
...                                    ...   
142.0   ESP_1                  2305.039248   
        ESP_1                  2303.593343   
        ESP_1                  2311.123677   
        ESP_1                  2285.897882   
        ESP_1                  2268.236267   

                ESP Data - Motor Temperature Shutdown Setpoint  \
Well_ID AL_Key                                                   
345.0   ESP_2                                      -509.534789   
        ESP_2                                     -2445.534789   
        ESP_2                                     -1629.534789   
        ESP_2                                     -2717.534789   
        ESP_2                                     -2293.534789   
...                                                        ...   
142.0   ESP_1                                       240.000000   
        ESP_1                                       240.000000   
        ESP_1                                       240.000000   
        ESP_1                                       240.000000   
        ESP_1                                       240.000000   

                ESP Data - Motor Winding Temperature  \
Well_ID AL_Key                                         
345.0   ESP_2                             187.439259   
        ESP_2                             188.548634   
        ESP_2                             190.544728   
        ESP_2                             179.779103   
        ESP_2                             186.111134   
...                                              ...   
142.0   ESP_1                             176.027185   
        ESP_1                             176.004112   
        ESP_1                             178.472401   
        ESP_1                             177.468553   
        ESP_1       

In [11]:
final_data = pd.concat([B60_daily_df, L60_daily_df],
    axis=0,
    join="outer",
    ignore_index=False,
    keys=None,
    levels=None,
    names=None,
    verify_integrity=False,
    copy=False,)

final_data = final_data.fillna(1)
final_data["Will_Fail"] = final_data["Will_Fail"].apply(np.int64)
final_data

OIL          GAS        WATER  DOWN_TIME_HOURS  \
Well_ID AL_Key                                                           
345.0   ESP_2    615.154601   650.447788   526.488655             11.0   
        ESP_2   1224.193640  1232.455787   809.414685              0.0   
        ESP_2   1007.769762  1047.068544   667.933899              0.0   
        ESP_2   1514.641876  1370.143160  1122.137652              0.0   
        ESP_2   1089.741764  1387.138130  1156.169599              0.0   
...                     ...          ...          ...              ...   
142.0   ESP_1    130.568357   335.639084   433.161488              0.0   
        ESP_1    135.061535   338.161005   421.979434              0.0   
        ESP_1    135.757477   341.732930   428.363835              0.0   
        ESP_1     64.788755   170.379740   221.002588             12.0   
        ESP_1      0.000000     0.000000     0.000000             24.0   

                ESP Data - Drive Current  ESP Data - Drive Voltage  \
Well_ID AL_Key                                                       
345.0   ESP_2                 249.860279                370.149250   
        ESP_2                 135.239185                464.211750   
        ESP_2                 146.440357                424.086750   
        ESP_2                 207.901295                505.961750   
        ESP_2                 153.452076                487.446125   
...                                  ...                       ...   
142.0   ESP_1                 232.000000                433.000000   
        ESP_1                 232.000000                433.000000   
        ESP_1                 230.433113                431.364779   
        ESP_1                 243.458937                423.202102   
        ESP_1                 369.099649                402.275272   

                ESP Data - Intake Pressure  \
Well_ID AL_Key                               
345.0   ESP_2                   368.715449   
        ESP_2                  -261.284551   
        ESP_2                  -200.909551   
        ESP_2                   222.215449   
        ESP_2                   303.371699   
...                                    ...   
142.0   ESP_1                   894.599976   
        ESP_1                   894.599976   
        ESP_1                   893.432975   
        ESP_1                   934.540501   
        ESP_1                  1527.854901   

                ESP Data - Motor Temperature Shutdown Setpoint  \
Well_ID AL_Key                                                   
345.0   ESP_2                                      -509.534789   
        ESP_2                                     -2445.534789   
        ESP_2                                     -1629.534789   
        ESP_2                                     -2717.534789   
        ESP_2                                     -2293.534789   
...                                                        ...   
142.0   ESP_1                                       240.000000   
        ESP_1                                       240.000000   
        ESP_1                                       240.000000   
        ESP_1                                       240.000000   
        ESP_1                                       240.000000   

                ESP Data - Motor Winding Temperature  \
Well_ID AL_Key                                         
345.0   ESP_2                             187.439259   
        ESP_2                             188.548634   
        ESP_2                             190.544728   
        ESP_2                             179.779103   
        ESP_2                             186.111134   
...                                              ...   
142.0   ESP_1                             182.800003   
        ESP_1                             182.800003   
        ESP_1                             182.836947   
        ESP_1                             186.748578   
        ESP_1       

In [12]:
corr_matrix = final_data.corr()
corr_matrix

,OIL,GAS,WATER,DOWN_TIME_HOURS,ESP Data - Drive Current,ESP Data - Drive Voltage,ESP Data - Intake Pressure,ESP Data - Motor Temperature Shutdown Setpoint,ESP Data - Motor Winding Temperature,ESP Data - Output Frequency,...,Pump_Delta_Pressure,Pump_Average_Pressure,Gas_Saturation_in_Pump,Pump_Power,Drive_Power,Power_Ratio,Power_Difference,ESP_Temperature,Lower_Limit,Will_Fail
OIL,1.000000,0.691848,0.632508,-0.135070,0.350139,0.112263,0.409533,0.113262,-0.065032,-0.271425,...,-0.230007,0.437420,0.230007,0.230007,0.458611,0.230007,-0.230007,-0.230007,0.233904,-0.523576
GAS,0.691848,1.000000,0.435645,-0.100728,0.163036,0.261546,0.223869,-0.029367,-0.050396,-0.025631,...,-0.136685,0.293392,0.136685,0.136685,0.317273,0.136685,-0.136685,-0.136685,0.174281,-0.338842
WATER,0.632508,0.435645,1.000000,-0.296724,0.547385,0.010645,0.653922,0.100695,-0.116230,-0.458633,...,-0.383813,0.707811,0.383813,0.383813,0.580459,0.383813,-0.383813,-0.383813,0.189431,-0.576275
DOWN_TIME_HOURS,-0.135070,-0.100728,-0.296724,1.000000,-0.010602,-0.069808,-0.039356,-0.063770,0.026836,-0.000295,...,0.057785,-0.090768,-0.057785,-0.057785,-0.051750,-0.057785,0.057785,0.057785,-0.004214,0.034253
ESP Data - Drive Current,0.350139,0.163036,0.547385,-0.010602,1.000000,0.142358,0.710282,-0.010274,-0.507298,-0.355106,...,-0.707828,0.668697,0.707828,0.707828,0.919133,0.707828,-0.707828,-0.707828,-0.020518,-0.299966
ESP Data - Drive Voltage,0.112263,0.261546,0.010645,-0.069808,0.142358,1.000000,-0.076834,-0.068530,-0.108789,0.386670,...,-0.270678,0.070318,0.270678,0.270678,0.318002,0.270678,-0.270678,-0.270678,0.013129,0.062004
ESP Data - Intake Pressure,0.409533,0.223869,0.653922,-0.039356,0.710282,-0.076834,1.000000,0.045195,-0.432743,-0.654574,...,-0.728788,0.882860,0.728788,0.728788,0.592739,0.728788,-0.728788,-0.728788,0.102282,-0.449923
ESP Data - Motor Temperature Shutdown Setpoint,0.113262,-0.029367,0.100695,-0.063770,-0.010274,-0.068530,0.045195,1.000000,0.149599,-0.033892,...,0.001010,0.012146,-0.001010,-0.001010,0.004900,-0.001010,0.001010,0.001010,0.118444,-0.061452
ESP Data - Motor Winding Temperature,-0.065032,-0.050396,-0.116230,0.026836,-0.507298,-0.108789,-0.432743,0.149599,1.000000,0.381051,...,0.628686,-0.418893,-0.628686,-0.628686,-0.347156,-0.628686,0.628686,0.628686,0.133748,0.079675
ESP Data - Output Frequency,-0.271425,-0.025631,-0.458633,-0.000295,-0.355106,0.386670,-0.654574,-0.033892,0.381051,1.000000,...,0.341931,-0.575244,-0.341931,-0.341931,-0.136525,-0.341931,0.341931,0.341931,-0.134652,0.415448


In [13]:
"""
sn.heatmap(corr_matrix, annot=False)
columns =  [ 'Gas_Intake', 'ESP Data - Intake Pressure']
import random
for i in range(10):
    
    well_ids = dynamic_df['Well_ID']
    id_ = random.choice(well_ids)
    df_filtered = dynamic_df[dynamic.index.unique() == id_]
    df_filtered[columns].plot()

plt.show()
"""

"\nsn.heatmap(corr_matrix, annot=False)\ncolumns =  [ 'Gas_Intake', 'ESP Data - Intake Pressure']\nimport random\nfor i in range(10):\n    \n    well_ids = dynamic_df['Well_ID']\n    id_ = random.choice(well_ids)\n    df_filtered = dynamic_df[dynamic.index.unique() == id_]\n    df_filtered[columns].plot()\n\nplt.show()\n"

In [14]:
X = final_data.loc[:, final_data.columns != 'Will_Fail']
y = final_data.loc[:, final_data.columns == 'Will_Fail']

from imblearn.over_sampling import SMOTE

os = SMOTE(random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
columns = X_train.columns

os_data_X,os_data_y = os.fit_resample(X_train, y_train)
os_data_X = pd.DataFrame(data=os_data_X,columns=columns )
os_data_y= pd.DataFrame(data=os_data_y,columns=['Will_Fail'])



In [16]:
final_data_vars = final_data.columns.values.tolist()

X=[i for i in final_data_vars if i not in y]



from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()
rfe = RFE(logreg, step = 20)
rfe = rfe.fit(os_data_X, os_data_y.values.ravel())
print(rfe.support_)
print(rfe.ranking_)

[False  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True False False False False False False False
 False False False False False False False False False False  True]
[2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1]


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [25]:
final_cols = []
for col, flag in zip(list(final_data.columns), rfe.support_):
    if flag:
        final_cols.append(col)
final_cols

['GAS',
 'WATER',
 'DOWN_TIME_HOURS',
 'ESP Data - Drive Current',
 'ESP Data - Drive Voltage',
 'ESP Data - Intake Pressure',
 'ESP Data - Motor Temperature Shutdown Setpoint',
 'ESP Data - Motor Winding Temperature',
 'ESP Data - Output Frequency',
 'ESP Data - Vibration X',
 'Startup_Count',
 'Oil_Intake',
 'Water_Intake',
 'Gas_Intake',
 'Liquid_Intake',
 'Gas_Saturation_at_Intake',
 'Lower_Limit']

GAS        WATER  DOWN_TIME_HOURS  \
Well_ID AL_Key                                              
345.0   ESP_2    650.447788   526.488655             11.0   
        ESP_2   1232.455787   809.414685              0.0   
        ESP_2   1047.068544   667.933899              0.0   
        ESP_2   1370.143160  1122.137652              0.0   
        ESP_2   1387.138130  1156.169599              0.0   
...                     ...          ...              ...   
142.0   ESP_1    335.639084   433.161488              0.0   
        ESP_1    338.161005   421.979434              0.0   
        ESP_1    341.732930   428.363835              0.0   
        ESP_1    170.379740   221.002588             12.0   
        ESP_1      0.000000     0.000000             24.0   

                ESP Data - Drive Current  ESP Data - Drive Voltage  \
Well_ID AL_Key                                                       
345.0   ESP_2                 249.860279                370.149250   
        ESP_2                 135.239185                464.211750   
        ESP_2                 146.440357                424.086750   
        ESP_2                 207.901295                505.961750   
        ESP_2                 153.452076                487.446125   
...                                  ...                       ...   
142.0   ESP_1                 232.000000                433.000000   
        ESP_1                 232.000000                433.000000   
        ESP_1                 230.433113                431.364779   
        ESP_1                 243.458937                423.202102   
        ESP_1                 369.099649                402.275272   

                ESP Data - Intake Pressure  \
Well_ID AL_Key                               
345.0   ESP_2                   368.715449   
        ESP_2                  -261.284551   
        ESP_2                  -200.909551   
        ESP_2                   222.215449   
        ESP_2                   303.371699   
...                                    ...   
142.0   ESP_1                   894.599976   
        ESP_1                   894.599976   
        ESP_1                   893.432975   
        ESP_1                   934.540501   
        ESP_1                  1527.854901   

                ESP Data - Motor Temperature Shutdown Setpoint  \
Well_ID AL_Key                                                   
345.0   ESP_2                                      -509.534789   
        ESP_2                                     -2445.534789   
        ESP_2                                     -1629.534789   
        ESP_2                                     -2717.534789   
        ESP_2                                     -2293.534789   
...                                                        ...   
142.0   ESP_1                                       240.000000   
        ESP_1                                       240.000000   
        ESP_1                                       240.000000   
        ESP_1                                       240.000000   
        ESP_1                                       240.000000   

                ESP Data - Motor Winding Temperature  \
Well_ID AL_Key                                         
345.0   ESP_2                             187.439259   
        ESP_2                             188.548634   
        ESP_2                             190.544728   
        ESP_2                             179.779103   
        ESP_2                             186.111134   
...                                              ...   
142.0   ESP_1                             182.800003   
        ESP_1                             182.800003   
        ESP_1                             182.836947   
        ESP_1                             186.748578   
        ESP_1                             224.843614   

                ESP Data - Output Frequency  ESP Data - Vibration X  \
Well_ID AL_Key                                               

In [43]:
# Select 2 predictor features here:
predictor_features = final_data[final_cols]           # for the first demonstration run we will use porosity and brittleness
#

response_feature = 'Will_Fail'

X = predictor_features                # extract the 2 selected response features, 500 samples to a X array
y = final_data[response_feature]  
X # extract selected response 500 samples to a response array

GAS        WATER  DOWN_TIME_HOURS  \
Well_ID AL_Key                                              
345.0   ESP_2    650.447788   526.488655             11.0   
        ESP_2   1232.455787   809.414685              0.0   
        ESP_2   1047.068544   667.933899              0.0   
        ESP_2   1370.143160  1122.137652              0.0   
        ESP_2   1387.138130  1156.169599              0.0   
...                     ...          ...              ...   
142.0   ESP_1    335.639084   433.161488              0.0   
        ESP_1    338.161005   421.979434              0.0   
        ESP_1    341.732930   428.363835              0.0   
        ESP_1    170.379740   221.002588             12.0   
        ESP_1      0.000000     0.000000             24.0   

                ESP Data - Drive Current  ESP Data - Drive Voltage  \
Well_ID AL_Key                                                       
345.0   ESP_2                 249.860279                370.149250   
        ESP_2                 135.239185                464.211750   
        ESP_2                 146.440357                424.086750   
        ESP_2                 207.901295                505.961750   
        ESP_2                 153.452076                487.446125   
...                                  ...                       ...   
142.0   ESP_1                 232.000000                433.000000   
        ESP_1                 232.000000                433.000000   
        ESP_1                 230.433113                431.364779   
        ESP_1                 243.458937                423.202102   
        ESP_1                 369.099649                402.275272   

                ESP Data - Intake Pressure  \
Well_ID AL_Key                               
345.0   ESP_2                   368.715449   
        ESP_2                  -261.284551   
        ESP_2                  -200.909551   
        ESP_2                   222.215449   
        ESP_2                   303.371699   
...                                    ...   
142.0   ESP_1                   894.599976   
        ESP_1                   894.599976   
        ESP_1                   893.432975   
        ESP_1                   934.540501   
        ESP_1                  1527.854901   

                ESP Data - Motor Temperature Shutdown Setpoint  \
Well_ID AL_Key                                                   
345.0   ESP_2                                      -509.534789   
        ESP_2                                     -2445.534789   
        ESP_2                                     -1629.534789   
        ESP_2                                     -2717.534789   
        ESP_2                                     -2293.534789   
...                                                        ...   
142.0   ESP_1                                       240.000000   
        ESP_1                                       240.000000   
        ESP_1                                       240.000000   
        ESP_1                                       240.000000   
        ESP_1                                       240.000000   

                ESP Data - Motor Winding Temperature  \
Well_ID AL_Key                                         
345.0   ESP_2                             187.439259   
        ESP_2                             188.548634   
        ESP_2                             190.544728   
        ESP_2                             179.779103   
        ESP_2                             186.111134   
...                                              ...   
142.0   ESP_1                             182.800003   
        ESP_1                             182.800003   
        ESP_1                             182.836947   
        ESP_1                             186.748578   
        ESP_1                             224.843614   

                ESP Data - Output Frequency  ESP Data - Vibration X  \
Well_ID AL_Key                                               

In [44]:
# Decison Tree Model with scikit learn
from sklearn import tree                                     # for accessing tree information

# Step 1. Instantiate the Model 
decision_tree_reg = tree.DecisionTreeRegressor(min_samples_leaf=5, max_depth = 3)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

n_train = len(X_train)
n_test = len(X_test)

# Step 2: Fit the Data on Training Data
decision_tree_reg.fit(X_train.values.reshape(n_train,2), y_train[response_feature]) # fit model

# Plot model fit
plt.subplot(121)
plt = visualize_model(decision_tree_reg,X_test[predictor_features[0]],X_test[predictor_features[1]],y_test[response_feature],'Testing Data and Decision Tree Model')
plt.subplots_adjust(left=0.0, bottom=0.0, right=1.2, top=1.2, wspace=0.3, hspace=0.2)

# Step 3: - Make predictions using the testing dataset
tree_y_pred = decision_tree_reg.predict(X_test.values.reshape(n_test,2))

# Report the goodness of fit
print('Variance explained: %.2f' % r2_score(y_test, tree_y_pred))

# Calculate the error at withheld testing samples
tree_y_res = tree_y_pred - y_test['Production'].values

plt.subplot(122)
plt.hist(tree_y_res, alpha = 0.2, color = 'red', edgecolor = 'black', bins=np.linspace(-1*fmax[rindex[0]],fmax[rindex[0]],40))
plt.title('Decision Tree Model Prediction Error - ' + response_feature + ' from ' + predictor_features[0] + ' and ' + predictor_features[1]); 
plt.xlabel(response_feature + ' Estimation Error '); plt.ylabel('Frequency')
plt.xlim(-1*fmax[rindex[0]],fmax[rindex[0]])

plt.subplots_adjust(left=0.0, bottom=0.0, right=3.2, top=1.6, wspace=0.1, hspace=0.2)
plt.show()


print(n_train)


ValueError: cannot reshape array of size 261358 into shape (15374,2)